In [8]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [6]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [7]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Chen is an exceptional instructor. Her lectures are engaging and she explains complex concepts clearly.'},
 {'professor': 'Prof. Michael Johnson',
  'subject': 'History',
  'stars': 4,
  'review': "Prof. Johnson's passion for history is contagious. His classes are always interesting, though sometimes a bit disorganized."},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Biology',
  'stars': 5,
  'review': "Dr. Thompson's labs are fantastic. She encourages critical thinking and makes difficult topics accessible."},
 {'professor': 'Prof. David Lee',
  'subject': 'Mathematics',
  'stars': 3,
  'review': 'Prof. Lee knows his subject well, but his teaching style can be dry at times. Extra office hours help.'},
 {'professor': 'Dr. Rachel Green',
  'subject': 'Psychology',
  'stars': 5,
  'review': 'Dr. Green is an inspiration. Her research-based approach and real-world examples make the class 

In [12]:
processed_data= []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [13]:
processed_data[0]

IndexError: list index out of range

In [14]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 05 Sep 2024 06:50:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '41', 'x-pinecone-request-id': '6888156634845944224', 'x-envoy-upstream-service-time': '39', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [ ]:
index.describe_index_stats()